##### Imports

In [12]:
from typing import Tuple
from pathlib import Path
from tqdm.notebook import tqdm

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from monai.networks import nets, one_hot

import plotly.express as px
import plotly.figure_factory as ff

from skimage import exposure

import torch
from torch import nn, optim
import torch.nn.functional as F
from torch.utils.data import Dataset, random_split, DataLoader
from torchvision import transforms

import torchio as tio

from kedro.extras.datasets.pandas import CSVDataSet
from kedro.extras.datasets.pickle import PickleDataSet

/opt/conda/envs/tagseg/lib/python3.8/site-packages/plotly/graph_objs/__init__.py:288: DeprecationWarning:

distutils Version classes are deprecated. Use packaging.version instead.



In [2]:
import os, sys
sys.path.append(os.path.abspath('../src'))

from tagseg.data import ScdEvaluator, MnmEvaluator
from tagseg.models.trainer import Trainer
from tagseg.models.segmenter import Net
from tagseg.pipelines.model_evaluation.nodes import tag_subjects
from tagseg.data.dmd_dataset import DmdDataSet

##### The rest

In [5]:
dsets = ['mnm', 'scd']
results = {ds: PickleDataSet(f'../data/03_primary/{ds}_test_tagged.pt').load() for ds in dsets}

results['dmd'] = PickleDataSet('../data/03_primary/dmd_test.pt').load()

In [52]:
np.array(list(zip(*[(subject.image.data.mean().item(), subject.image.data.std().item()) for subject in results['dmd']]))).mean(axis=1)

array([-1.14303447,  0.74665013])

In [53]:
np.array(list(zip(
    *[((0.5 * subject.image.data + 1.0).mean().item(), (0.5 * subject.image.data + 1.0).std().item()) 
        for subject in results['dmd']]))).mean(axis=1)

array([0.42848276, 0.37332507])

In [50]:
np.array(list(zip(*[(subject.image.data.mean().item(), subject.image.data.std().item()) for subject in results['mnm']]))).mean(axis=1)

array([0.72535978, 0.18047594])

In [51]:
np.array(list(zip(*[(subject.image.data.mean().item(), subject.image.data.std().item()) for subject in results['scd']]))).mean(axis=1)

array([0.69256818, 0.14355787])